# Numerische Überlegungen zur Summenbildung

Wir verwenden 32-Bit Floats (Single Precision Floating Point) siehe [hier](https://en.wikipedia.org/wiki/Single-precision_floating-point_format).

Der folgende Code erstellt ein Array mit 1 Million und 1 Eintrag:
- 1 Million mal die 1.0
- 1 mal die 1,000,000,000,000.0 (1e12)

Das Array `values_asc` ist aufsteigend sortiert. Es wird ebenso eine absteigend sortierte Version `values_desc` und eine zufällig angeordnete Version `values_rand` erstellt. 

Von allen Arrays wird mit der `.sum()` -Methode die Summe gebildet. Welche werten kommen raus? Warum unterscheiden sich diese? Danach kommt eine manuelle For-Schleife - wie vergleicht sich das Ergebnis. Können Sie in NumPy-Dokumentation rausfinden welches Verfahren hier genutzt wird?

Implementieren Sie zwei geeignetere Verfahren:
- Verwendung einer 64-Bit Float Variable für die Summe
- [Paarweise Addition](https://en.wikipedia.org/wiki/Pairwise_summation)

Welche Ergebnisse erhalten Sie? Warum sind diese genauer?

In [73]:
import numpy as np

In [67]:
# Erstelle ein Array mit 1 Million mal 1.0 und 1 Eintrag 1e12
values_asc = np.array(int(1e6) * [1.0] + [1e12],dtype=np.float32)
values_asc

array([1.e+00, 1.e+00, 1.e+00, ..., 1.e+00, 1.e+00, 1.e+12], dtype=float32)

In [74]:
# Setze den Eintrag für 1e12 an die erste Stelle
values_desc = np.flip(values_asc)
values_desc

array([1.e+12, 1.e+00, 1.e+00, ..., 1.e+00, 1.e+00, 1.e+00], dtype=float32)

In [75]:
# Zufällige Reihenfolge
np.random.seed(2021)
values_rand = values_asc.copy()
np.random.shuffle(values_rand)
values_rand

array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)

In [95]:
# Summen:
print(f'Ascending order:  {values_asc.sum():18,.0f}')
print(f'Random order:     {values_rand.sum():18,.0f}')
print(f'Descending order: {values_desc.sum():18,.0f}')

Ascending order:   1,000,000,978,944
Random order:      1,000,000,389,120
Descending order:    999,999,995,904


In [96]:
total = np.float32(0)
for v in values_desc:
    total += v
print(f'For-loop desc:    {total:18,.0f}')

For-loop desc:       999,999,995,904


In [97]:
total = np.float64(0.0)
for v in values_desc:
    total += v
print(f'Float64:  {total:18,.0f}')

Float64:   1,000,000,995,904


In [91]:
def pairwise(array):
    if len(array) == 1:
        return array[0]
    else:
        return pairwise(array[:len(array) // 2]) + pairwise(array[len(array) // 2:])

In [92]:
pairwise(values_asc)

1000001000000.0

In [93]:
pairwise(values_desc)

1000001000000.0

In [94]:
pairwise(values_rand)

1000001000000.0

In [98]:
a = 1e20
a

1e+20

In [100]:
b = 1
b

1

In [101]:
a + b

1e+20